<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GroceryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install faker

In [11]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/GroceryPrices.xlsx -O GroceryPrices.xlsx

In [12]:
import requests
from lxml import html
import numpy as np
import pandas as pd
from collections import deque
from faker import Faker
import plotly.express as px

In [13]:
idx=pd.IndexSlice

In [14]:
rng=np.random.RandomState()

In [15]:
df_grocery = pd.read_excel('GroceryPrices.xlsx')

In [16]:
fake=Faker()
names=[fake.name_nonbinary() for i in range(100)]

In [54]:
now = pd.to_datetime('now').round('s')
df_names=[]
for _ in range(50):
    starts = now + pd.to_timedelta(rng.randint(100,24*3600*30,size=rng.poisson(3)+1),'s')
    for name in rng.choice(names,rng.poisson(0.5)+1):
        df_start=[]
        for start in starts:
            n_items = rng.poisson(10)+1
            times = start + pd.to_timedelta(np.cumsum(rng.poisson(10,n_items))+1,'s')
            items = rng.choice(df_grocery['Item'],n_items)
            quantity = rng.poisson(0.5, size=n_items)+1
            df_start.append(pd.DataFrame(dict(time=times, item=items,quantity=quantity)))

        df_name=pd.concat(df_start)
        df_name['name']=name
        df_names.append(df_name)
df_noprices=pd.concat(df_names)

In [55]:
df_withPrice=df_noprices.sort_values('time')

In [57]:
df_withPrice=df_noprices.merge(df_grocery, left_on='item', right_on='Item').drop('Item',axis=1)
df_withPrice['subtotal']=df_withPrice['Price']*df_withPrice['quantity']

In [58]:
df_shopping=df_withPrice.sort_values(by=['name','time'])

In [59]:
def trip_count(d):
    d['trip']=np.cumsum(d['time'].diff()>pd.to_timedelta(60,'s'))
    return d 

In [60]:
df_trip = df_shopping.groupby('name').apply(trip_count)

In [61]:
def checkout_duration_total(d):
    return pd.Series(dict(duration=(pd.Timedelta(d['time'].max()-d['time'].min()))/pd.to_timedelta(1,'s'),
                          total=d['subtotal'].sum()))

In [62]:
df_checkout=df_trip.groupby(['name','trip']).apply(checkout_duration_total)

In [63]:
df_first16_names=df_checkout.loc[idx[:df_checkout.index.levels[0][15],:],:].reset_index()

In [64]:
fig=px.bar(df_first16_names,x='trip',y='duration',facet_col='name',facet_col_wrap=4, template='plotly_dark',height=800)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("name=","")))

In [65]:
fig2=px.bar(df_first16_names,x='trip',y='total',facet_col='name',facet_col_wrap=4, template='plotly_dark',height=800)
fig2.for_each_annotation(lambda a: a.update(text=a.text.replace("name=","")))